<a href="https://colab.research.google.com/github/hmghaly/hparser/blob/master/Parsing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
cwd="/content/drive/My Drive/Research/code"
os.chdir(cwd)

Mounted at /content/drive/


In [ ]:
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 3819, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 3819 (delta 64), reused 0 (delta 0), pack-reused 3709 (from 1)
Receiving objects: 100% (3819/3819), 29.35 MiB | 13.23 MiB/s, done.
Resolving deltas: 100% (2134/2134), done.


#Parsing flow
connect to rules and features tables - build parsing algorithm

In [2]:
from code_utils.pandas_utils import *
from itertools import groupby

rules_workbook_url="https://docs.google.com/spreadsheets/d/e/2PACX-1vQWwi-tdS4z1oXN3fELGqBSh6elDa2OtTIh7zFLZfufOON2PYIg2QBd9avPdXz2g3CwpmCj-tNV76Qn/pub?output=xlsx"
rw_data_dict=get_wb_data_dict(rules_workbook_url)

rw_data_dict.keys()
rules_list=rw_data_dict["short-list"]
word_features_list=rw_data_dict["words-features"]

for a in  rules_list:
  print(a)
word_features_list.sort(key=lambda x:x["word"])
for a in word_features_list:
  if a["category"]=="": continue
  #print(a)

word_features_list_grouped=[(key,[(v["category"],v["features"]) for v in list(group)]) for key,group in groupby(word_features_list,lambda x:x["word"])]
# for a in word_features_list_grouped:
#   print(a)

final_word_features_dict=dict(iter(word_features_list_grouped))

{'Parent': 'S', 'Rule': 'S[+declarative]  → NP VP', 'Description': '', 'Example': '', 'Unnamed: 4': '', 'Unnamed: 5': '1', 'Unnamed: 6': 'S[+declarative]  → NP VP', 'Unnamed: 7': 'declarative sentence', 'Unnamed: 8': 'The boy ate the apple', 'Unnamed: 9': '', 'Unnamed: 10': 'Non-Terminal', 'Unnamed: 11': 'Description', 'Unnamed: 12': 'Example', 'Unnamed: 13': '', 'Unnamed: 14': 'Non-Terminal'}
{'Parent': 'S', 'Rule': 'S  → NP VP^', 'Description': '', 'Example': 'the man is reading', 'Unnamed: 4': '', 'Unnamed: 5': '2', 'Unnamed: 6': 'S[+YNQ] → Aux S/Aux', 'Unnamed: 7': 'Yes/No Question', 'Unnamed: 8': 'Did he eat the apple?', 'Unnamed: 9': '', 'Unnamed: 10': 'S  → NP VP^', 'Unnamed: 11': '', 'Unnamed: 12': 'the man is reading', 'Unnamed: 13': '', 'Unnamed: 14': 'NP → XP[+DET] + NP[-DET]'}
{'Parent': 'S', 'Rule': 'S-YNQ → Aux^ S/Aux', 'Description': '', 'Example': 'are you available?', 'Unnamed: 4': '', 'Unnamed: 5': '3', 'Unnamed: 6': 'S[+WHQ] → WH + Aux + S/Aux/PP', 'Unnamed: 7': 'WH-

##Process rules

In [3]:
import re

# def process_rule(rule_str):
#   final_rule_dict={}
#   rule_str=rule_str.replace("→","-->")
#   rule_split=rule_str.split("-->")
#   lhs,rhs=rule_split
#   lhs,rhs=lhs.strip(),rhs.strip()
#   lhs_cat0=lhs.split("[")[0]
#   lhs_features=re.findall(r'\[(.+?)\]',lhs)
#   cur_rhs_features=[]
#   for ft0 in lhs_features:
#     cur_rhs_features.extend(ft0.split())
#   lhs_dict={"cat":lhs_cat0,"features":cur_rhs_features}
#   final_rule_dict["parent"]=lhs_dict
#   rule_children=[]
#   rhs_items=rhs.split()
#   for it0 in rhs_items:
#     item_dict={}
#     is_head=False
#     features=re.findall(r'\[(.+?)\]',it0)
#     if "^" in it0: is_head=True
#     it0=it0.replace("^","")
#     item_cat0=it0.split("[")[0]
#     cur_features=[]
#     for ft0 in features:
#       cur_features.extend(ft0.split())

#     item_dict={"cat":item_cat0,"is_head":is_head,"features":cur_features}
#     rule_children.append(item_dict)
#   final_rule_dict["children"]=rule_children
#   return final_rule_dict

rules=["NP --> X[+noun]",
       "AP --> JJ",
       "VP --> X^[+be] AP",
       "S --> NP VP^",
       "V → V[+be] V^[+ing]",
       'Aux[+neg] → Aux^ RB[+neg]'
       ]
# all_processed_rules=[]
# cat_fwd_index=[]
# feat_fwd_index=[]
# for i0,r0 in enumerate(rules):
#   print(r0)
#   processed_rule=process_rule(r0)
#   all_processed_rules.append(processed_rule)
#   last_child=processed_rule["children"][-1]
#   last_child_cat=last_child["cat"]
#   last_child_features=last_child["features"]
#   cat_fwd_index.append((last_child_cat,i0))
#   for ft0 in last_child_features: feat_fwd_index.append((ft0,i0))

#   print(processed_rule)
#   print("last_child",last_child)
#   print("-----")

# cat_fwd_index.sort()
# print("cat_fwd_index",cat_fwd_index)
# feat_fwd_index.sort()
# print("feat_fwd_index",feat_fwd_index)

# cat_inv_index=dict(iter([(key,[v[1] for v in list(group)]) for key,group in groupby(cat_fwd_index,lambda x:x[0])]))
# print("cat_inv_index",cat_inv_index)

NP --> X[+noun]
{'parent': {'cat': 'NP', 'features': []}, 'children': [{'cat': 'X', 'is_head': False, 'features': ['+noun']}]}
last_child {'cat': 'X', 'is_head': False, 'features': ['+noun']}
-----
AP --> JJ
{'parent': {'cat': 'AP', 'features': []}, 'children': [{'cat': 'JJ', 'is_head': False, 'features': []}]}
last_child {'cat': 'JJ', 'is_head': False, 'features': []}
-----
VP --> X^[+be] AP
{'parent': {'cat': 'VP', 'features': []}, 'children': [{'cat': 'X', 'is_head': True, 'features': ['+be']}, {'cat': 'AP', 'is_head': False, 'features': []}]}
last_child {'cat': 'AP', 'is_head': False, 'features': []}
-----
S --> NP VP^
{'parent': {'cat': 'S', 'features': []}, 'children': [{'cat': 'NP', 'is_head': False, 'features': []}, {'cat': 'VP', 'is_head': True, 'features': []}]}
last_child {'cat': 'VP', 'is_head': True, 'features': []}
-----
V → V[+be] V^[+ing]
{'parent': {'cat': 'V', 'features': []}, 'children': [{'cat': 'V', 'is_head': False, 'features': ['+be']}, {'cat': 'V', 'is_head': Tr

In [16]:
class Parser:
  def __init__(self,rules_list=[],word_features_list=[],params={}) -> None:
    self.unknown_tags=params.get("unknown_tags",["N","V","JJ","RB"]) #maybe get the actual distribution of these tags from corpora
    self.sent_padding=params.get("sent_padding",False)
    #process all word features

    #process all rules
    self.all_processed_rules=[]
    self.cat_fwd_index=[]
    self.feat_fwd_index=[]
    for i0,r0 in enumerate(rules_list):
      processed_rule=process_rule(r0)
      #print(processed_rule)
      self.all_processed_rules.append(processed_rule)
      last_child=processed_rule["children"][-1]
      last_child_cat=last_child["cat"]
      last_child_features=last_child["features"]
      self.cat_fwd_index.append((last_child_cat,i0))
      for ft0 in last_child_features: self.feat_fwd_index.append((ft0,i0))
    self.cat_fwd_index.sort()
    self.feat_fwd_index.sort()
    self.cat_inv_index=dict(iter([(key,[v[1] for v in list(group)]) for key,group in groupby(self.cat_fwd_index,lambda x:x[0])]))
    self.feat_inv_index=dict(iter([(key,[v[1] for v in list(group)]) for key,group in groupby(self.feat_fwd_index,lambda x:x[0])]))

  def parse(self,tokens):
    phrase_list=[]
    cat_phrase_index={}
    feat_phrase_index={}
    for i,a in enumerate(tokens):
      outcome=final_word_features_dict.get(a)
      if outcome==None:
        pass
      else:
        pass
      print(i,a, outcome)

def process_rule(rule_str):
  final_rule_dict={}
  rule_str=rule_str.replace("→","-->")
  rule_split=rule_str.split("-->")
  lhs,rhs=rule_split
  lhs,rhs=lhs.strip(),rhs.strip()
  lhs_cat0=lhs.split("[")[0]
  lhs_features=re.findall(r'\[(.+?)\]',lhs)
  cur_rhs_features=[]
  for ft0 in lhs_features:
    cur_rhs_features.extend(ft0.split())
  lhs_dict={"cat":lhs_cat0,"features":cur_rhs_features}
  final_rule_dict["parent"]=lhs_dict
  rule_children=[]
  rhs_items=rhs.split()
  for it0 in rhs_items:
    item_dict={}
    is_head=False
    features=re.findall(r'\[(.+?)\]',it0)
    if "^" in it0: is_head=True
    it0=it0.replace("^","")
    item_cat0=it0.split("[")[0]
    cur_features=[]
    for ft0 in features:
      cur_features.extend(ft0.split())

    item_dict={"cat":item_cat0,"is_head":is_head,"features":cur_features}
    rule_children.append(item_dict)
  final_rule_dict["children"]=rule_children
  return final_rule_dict

rules=["NP --> X[noun]",
       "AP --> JJ",
       "VP --> X^[be] AP",
       "S --> NP VP^",
       "V → V[be] V^[ing]",
       'Aux[neg] → Aux^ RB[neg]'
       ]



sent="he is happy"
unknown_tags=["N","V","JJ","RB"]
sent_toks=sent.split()
print(sent_toks)

parser_obj=Parser(rules_list=rules)
parser_obj.parse(sent_toks)
# for i,a in enumerate(sent_toks):
#   outcome=final_word_features_dict.get(a)
#   if outcome==None:
#     pass
#   else:
#     pass
#   print(i,a, outcome)

['he', 'is', 'happy']
0 he [('PRON', 'noun 3rd masc sentient|human')]
1 is [('AUX', 'be 3rd singular present')]
2 happy None


In [15]:
parser_obj.feat_fwd_index

[('ing', 4), ('neg', 5), ('noun', 0)]

In [ ]:
#parse each rule - identify head of the rule, and feature application, including slash
#convert into a trie
#collect word features

#for an input sentence:
#tokenize the sentence with tok paradigm
#locate/retrieve the corresponding features for each word, and word clusters (multi token words)
#for each word, generate the possible labels/tags, with their likelihood, if available
#a simple POS tagger for any given word

#A simple POS tagger, based on training data from multiple treebanks
#we need to resolve POS/tags, which to use, and how it contributes to different features NNS, VBG .. etc


#UD Treebanks

In [3]:
#process treebanks for POS info
#also extract dependency pairs to use for reranking
#also extract features
#build terminal rules based on standard/universal POS tags
#adjust the features in our rule tables, according to universal features

def iter_treebank(treebank_fpath):
      fopen=open(cur_train_file)
      cur_conll_str=""
      for i0,line0 in enumerate(fopen):
        if line0.startswith("#"): continue
        if line0.strip()=="":
          yield cur_conll_str
          cur_conll_str=""
        else:

          line_split0=line0.strip().split("\t")
          id0,form0,lemma0,upos0,xpos0,feats0,head0,deprel0,deps0,misc0=line_split0[:10]
          if not id0.isdigit(): continue
          cur_conll_str+=line0
      fopen.close()

word_upos_dict={}
word_xpos_dict={}
word_feat_dict={}


ud_dir="../data/UD"
excluded_tb_list=['UD_English-GENTLE']
treebank_dir_list=os.listdir(ud_dir)
for tb0 in treebank_dir_list:
  #if tb0 in excluded_tb_list: continue
  tb_full_path=os.path.join(ud_dir,tb0)
  #print(tb_full_path)
  tb_files=os.listdir(tb_full_path)
  for tbf0 in tb_files:
    if "train" in tbf0.lower() and "conllu" in tbf0.lower():
      cur_train_file=os.path.join(tb_full_path,tbf0)
      print(cur_train_file)
      #
      fopen=open(cur_train_file)
      cur_conll_str=""
      for i0,line0 in enumerate(fopen):
        if line0.startswith("#"): continue
        if line0.strip()=="":
          #print(cur_conll_str)
          #print("======")
          cur_conll_str=""
        else:
          cur_conll_str+=line0
          line_split0=line0.strip().split("\t")
          id0,form0,lemma0,upos0,xpos0,feats0,head0,deprel0,deps0,misc0=line_split0[:10]
          if not id0.isdigit(): continue
          temp_word_upos_dict=word_upos_dict.get(form0,{})
          temp_word_upos_dict[upos0]=temp_word_upos_dict.get(upos0,0)+1
          word_upos_dict[form0]=temp_word_upos_dict

          temp_word_xpos_dict=word_xpos_dict.get(form0,{})
          temp_word_xpos_dict[xpos0]=temp_word_xpos_dict.get(xpos0,0)+1
          word_xpos_dict[form0]=temp_word_xpos_dict

          temp_word_feat_dict=word_feat_dict.get(form0,{})
          temp_word_feat_dict[feats0]=temp_word_feat_dict.get(feats0,0)+1
          word_feat_dict[form0]=temp_word_feat_dict

          #word_feat_dict


          #print(line_split0)
        #print(line0.strip())
        #if i0>300: break
      fopen.close()

items=list(word_upos_dict.items())
items.sort(key=lambda x:-sum(x[1].values()))
for a,b in items[:500]:
  print(a,b)
  print(word_xpos_dict[a])
  print(word_feat_dict[a])
  print("------")


  #print()

#!ls "../data/UD"

../data/UD/UD_English-EWT/en_ewt-ud-train.conllu
../data/UD/UD_English-CHILDES/en_childes-ud-train.conllu
../data/UD/UD_English-Atis/en_atis-ud-train.conllu
../data/UD/UD_English-ESLSpok/en_eslspok-ud-train.conllu
../data/UD/UD_English-ParTUT/en_partut-ud-train.conllu
../data/UD/UD_English-LinES/en_lines-ud-train.conllu
../data/UD/UD_English-GUMReddit/en_gumreddit-ud-train.conllu
../data/UD/UD_English-GUM/en_gum-ud-train.conllu
. {'PUNCT': 46459}
{'.': 41753, ',': 10, 'FS': 1622, 'FF': 5, 'Period': 3068, ':': 1}
{'_': 46456, 'Typo=Yes': 3}
------
the {'DET': 28319, 'PRON': 7, 'ADP': 2, 'PART': 2, 'ADV': 1, 'X': 1}
{'DT': 21014, 'WDT': 1, 'PRP': 6, 'IN': 2, 'TO': 2, '_': 1746, 'RD': 2307, 'DEF': 3253, 'GW': 1}
{'Definite=Def|PronType=Art': 21157, 'PronType=Rel|Typo=Yes': 1, 'Case=Nom|Number=Plur|Person=3|PronType=Prs|Typo=Yes': 6, 'Typo=Yes': 4, '_': 5417, 'PronType=Art': 1745, 'Definite=Def|Number=Sing|PronType=Art': 1, 'Definite=Def|PronType=Art|Typo=Yes': 1}
------
, {'PUNCT': 22949,

In [20]:
word="so"
word_upos_dict[word], word_xpos_dict[word], word_feat_dict[word]


({'ADV': 1125, 'SCONJ': 191, 'ADP': 1, 'CCONJ': 45, 'INTJ': 28},
 {'RB': 1005, 'IN': 149, '_': 123, 'CC': 45, 'UH': 28, 'B': 27, 'CS': 13},
 {'_': 1305, 'ExtPos=SCONJ': 82, 'Degree=Pos': 2, 'Typo=Yes': 1})

In [ ]:
!head -150 ../data/UD/UD_English-EWT/en_ewt-ud-train.conllu

# newdoc id = weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000
# sent_id = weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000-0001
# newpar id = weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000-p0001
# text = Al-Zaman : American forces killed Shaikh Abdullah al-Ani, the preacher at the mosque in the town of Qaim, near the Syrian border.
1	Al	Al	PROPN	NNP	Number=Sing	0	root	0:root	SpaceAfter=No
2	-	-	PUNCT	HYPH	_	3	punct	3:punct	SpaceAfter=No
3	Zaman	Zaman	PROPN	NNP	Number=Sing	1	flat	1:flat	_
4	:	:	PUNCT	:	_	7	punct	7:punct	_
5	American	American	ADJ	JJ	Degree=Pos	6	amod	6:amod	_
6	forces	force	NOUN	NNS	Number=Plur	7	nsubj	7:nsubj	_
7	killed	kill	VERB	VBD	Mood=Ind|Number=Plur|Person=3|Tense=Past|VerbForm=Fin	1	parataxis	1:parataxis	_
8	Shaikh	Shaikh	PROPN	NNP	Number=Sing	7	obj	7:obj	_
9	Abdullah	Abdullah	PROPN	NNP	Number=Sing	8	flat	8:flat	_
10	al	al	PROPN	NNP	Number=Sing	8	flat	8:flat	SpaceAfter=No
11	-	-	PUNCT	HYPH	_	12	punct	12:punct	SpaceAf

In [ ]:
fpath="../data/UD/UD_English-EWT/en_ewt-ud-train.conllu"

word_xpos_list=[]
word_upos_list=[]

word_upos_dict={}
word_xpos_dict={}
word_feat_dict={}


fopen=open(fpath)
cur_conll_str=""
for i0,line0 in enumerate(fopen):
  if line0.startswith("#"): continue
  if line0.strip()=="":
    #print(cur_conll_str)
    #print("======")
    cur_conll_str=""
  else:
    cur_conll_str+=line0
    line_split0=line0.strip().split("\t")
    id0,form0,lemma0,upos0,xpos0,feats0,head0,deprel0,deps0,misc0=line_split0[:10]
    if not id0.isdigit(): continue
    temp_word_upos_dict=word_upos_dict.get(form0,{})
    temp_word_upos_dict[upos0]=temp_word_upos_dict.get(upos0,0)+1
    word_upos_dict[form0]=temp_word_upos_dict

    temp_word_xpos_dict=word_xpos_dict.get(form0,{})
    temp_word_xpos_dict[xpos0]=temp_word_xpos_dict.get(xpos0,0)+1
    word_xpos_dict[form0]=temp_word_xpos_dict

    temp_word_feat_dict=word_feat_dict.get(form0,{})
    temp_word_feat_dict[feats0]=temp_word_feat_dict.get(feats0,0)+1
    word_feat_dict[form0]=temp_word_feat_dict

    #word_feat_dict


    #print(line_split0)
  #print(line0.strip())
  #if i0>300: break
fopen.close()

items=list(word_upos_dict.items())
items.sort(key=lambda x:-sum(x[1].values()))
for a,b in items[:300]:
  print(a,b,word_xpos_dict[a],word_feat_dict[a])

. {'PUNCT': 8640} {'.': 8632, ',': 8} {'_': 8640}
the {'DET': 8141, 'PRON': 7, 'ADP': 2, 'PART': 1} {'DT': 8141, 'WDT': 1, 'PRP': 6, 'IN': 2, 'TO': 1} {'Definite=Def|PronType=Art': 8141, 'PronType=Rel|Typo=Yes': 1, 'Case=Nom|Number=Plur|Person=3|PronType=Prs|Typo=Yes': 6, 'Typo=Yes': 3}
, {'PUNCT': 7019, 'X': 2} {',': 7018, 'GW': 2, '.': 1} {'_': 7021}
to {'PART': 3227, 'ADP': 1785, 'SCONJ': 54, 'ADV': 6, 'VERB': 1, 'DET': 1, 'X': 1, 'NOUN': 1} {'TO': 3227, 'IN': 1840, 'RB': 6, 'VB': 1, 'DT': 1, 'NN': 1} {'_': 5067, 'Typo=Yes': 6, 'Typo=Yes|VerbForm=Inf': 1, 'Definite=Def|PronType=Art|Typo=Yes': 1, 'Number=Sing': 1}
and {'CCONJ': 4843, 'DET': 4, 'X': 6, 'ADP': 2} {'CC': 4843, 'DT': 4, 'GW': 6, 'IN': 2} {'_': 4849, 'Definite=Ind|PronType=Art|Typo=Yes': 4, 'Typo=Yes': 2}
a {'DET': 3589, 'NUM': 4, 'ADP': 7, 'NOUN': 4, 'ADV': 2, 'CCONJ': 1, 'PART': 1, 'AUX': 1} {'DT': 3589, 'LS': 4, 'IN': 7, 'NN': 4, 'RB': 2, 'CC': 1, 'TO': 1, 'MD': 1} {'Definite=Ind|PronType=Art': 3589, '_': 5, 'Abbr=Yes'

In [ ]:
treebank_fpath="../data/UD/UD_English-EWT/en_ewt-ud-train.conllu"
cur_iterator=iter_treebank(treebank_fpath)

for i0,a0 in enumerate(cur_iterator):
  print(a0)
  print("========")
  if i0>20: break

1	Aesthetic	aesthetic	ADJ	JJ	Degree=Pos	2	amod	2:amod	Discourse=organization-heading:1->57:8:grf-ly-_-_+sem-lxchn-1,671-_+sem-lxchn-4,619-_+sem-lxchn-5,620-_|Entity=(1-abstract-new-nnnnn-cf1-2-sgl|MSeg=Aesthet-ic
2	Appreciation	appreciation	NOUN	NN	Number=Sing	0	root	0:root	Entity=1)|MSeg=Appreciat-ion
3	and	and	CCONJ	CC	_	5	cc	5:cc	_
4	Spanish	Spanish	ADJ	JJ	Degree=Pos	5	amod	5:amod	Entity=(2-abstract-new-snssn-cf2-2-coref|MSeg=Span-ish
5	Art	art	NOUN	NN	Number=Sing	2	conj	2:conj:and	Entity=2)|SpaceAfter=No
6	:	:	PUNCT	:	_	2	punct	2:punct	_

1	Insights	insight	NOUN	NNS	Number=Plur	0	root	0:root	Discourse=elaboration-additional:2->1:0:grf-col-6-gold|Entity=(3-abstract-new-nnnns-cf1-1-coref|MSeg=In-sigh-t-s
2	from	from	ADP	IN	_	5	case	5:case	_
3	Eye	eye	NOUN	NN	Number=Sing	5	compound	5:compound	Entity=(4-abstract-new-sssss-cf2-3-coref(5-object-new-sssss-cf3-1-coref)|SpaceAfter=No|XML=<w>
4	-	-	PUNCT	HYPH	_	3	punct	3:punct	SpaceAfter=No
5	Tracking	tracking	NOUN	NN	Number=Sing	1	nmod	1:nm

#Other Parsers

In [1]:
#https://stanfordnlp.github.io/stanza/#getting-started
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 21.7 MB/s eta 0:00:00


In [2]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[
  [
    {
      "id": 1,
      "text": "Barack",
      "lemma": "Barack",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 4,
      "deprel": "nsubj:pass",
      "start_char": 0,
      "end_char": 6,
      "ner": "B-PERSON",
      "multi_ner": [
        "B-PERSON"
      ]
    },
    {
      "id": 2,
      "text": "Obama",
      "lemma": "Obama",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 1,
      "deprel": "flat",
      "start_char": 7,
      "end_char": 12,
      "ner": "E-PERSON",
      "multi_ner": [
        "E-PERSON"
      ]
    },
    {
      "id": 3,
      "text": "was",
      "lemma": "be",
      "upos": "AUX",
      "xpos": "VBD",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "head": 4,
      "deprel": "aux:pass",
      "start_char": 13,
      "end_char": 16,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "b

In [5]:
#doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence

sent="To support this integration, my Office has developed practical tools, including the newly launched third edition of the Aide-memoire on options for reflecting weapons and ammunition management in decisions of the Security Council and an online dashboard, which provide guidance for the drafting of resolutions to effectively include small arms and ammunition considerations."
doc = nlp(sent) # run annotation over a sentence

print(doc)
#print(doc.entities)

[
  [
    {
      "id": 1,
      "text": "To",
      "lemma": "to",
      "upos": "PART",
      "xpos": "TO",
      "head": 2,
      "deprel": "mark",
      "start_char": 0,
      "end_char": 2,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "support",
      "lemma": "support",
      "upos": "VERB",
      "xpos": "VB",
      "feats": "VerbForm=Inf",
      "head": 9,
      "deprel": "advcl",
      "start_char": 3,
      "end_char": 10,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "this",
      "lemma": "this",
      "upos": "DET",
      "xpos": "DT",
      "feats": "Number=Sing|PronType=Dem",
      "head": 4,
      "deprel": "det",
      "start_char": 11,
      "end_char": 15,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "integration",
      "lemma": "integration",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "N

In [ ]:
#https://lindat.mff.cuni.cz/services/udpipe/
#https://ufal.mff.cuni.cz/udpipe/2
